In [11]:
from google.colab import drive

In [12]:
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!ls "/content/drive/My Drive"

 algorithmProject   Test	      Untitled1.ipynb
'Deep Learning'     Untitled0.ipynb   Untitled2.ipynb


In [14]:
!ls "/content/drive/My Drive/Deep Learning"

 1.jpg	'Assignment #4.ipynb'   Untitled	  Untitled2.ipynb   wine1.xls
 2.jpg	'Assignment #5.ipynb'   Untitled0.ipynb   Untitled3.ipynb
 3.jpg	 cifar-10-batches-py    Untitled1.ipynb   win2.xls


In [15]:
!ls '/content/drive/My Drive/Deep Learning/cifar-10-batches-py'

batches.meta  data_batch_2  data_batch_4  desktop.ini  test_batch
data_batch_1  data_batch_3  data_batch_5  readme.html


In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

import keras as k
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
torch.__version__

'1.6.0+cu101'

In [4]:
from torch.autograd import Variable
use_gpu = torch.cuda.is_available()
use_gpu

True

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols, 3)
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

In [8]:
def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000,3072)
        Y = np.array(Y)
        return X, Y

In [9]:
def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [41]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
    # Load the raw CIFAR-10 data
    cifar10_dir = r'/content/drive/My Drive/Deep Learning/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    x_train = X_train.astype('float32')
    x_test = X_test.astype('float32')

  
    return x_train, y_train, X_val, y_val, x_test, y_test
# Invoke the above function to get our data.
x_train, y_train, x_val, y_val, x_test, y_test = get_CIFAR10_data()

In [42]:
print('Train data shape: ', x_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', x_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 3072)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3072)
Validation labels shape:  (1000,)
Test data shape:  (10000, 3072)
Test labels shape:  (10000,)


In [32]:
from __future__ import print_function

In [35]:
model = Sequential()
batch_size = 64
num_classes = 10
epochs = 50

In [43]:
x_train_scaled = x_train / 255
x_test_scaled = x_test / 255

In [47]:
y_train_categorical = keras.utils.to_categorical(y_train, num_classes= 10, dtype='float')
y_test_categorical = keras.utils.to_categorical(y_test, num_classes= 10, dtype='float')

y_train_categorical[:5]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [44]:
def create_model():
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(32,32,3)),
        keras.layers.Dense(3000, activation='relu'),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(10, activation='sigmoid')
    ])

    model.compile(optimizer= 'SGD',
                 loss= 'categorical_crossentropy',
                 metrics=['accuracy'])
    
    return model

In [48]:
from tensorflow import keras

model = create_model()
model.fit(x_train_scaled, y_train_categorical, epochs=50)

Epoch 1/50
1532/1532 [==============================] - 5s 3ms/step - loss: 1.8669 - accuracy: 0.3283
Epoch 2/50
1532/1532 [==============================] - 5s 3ms/step - loss: 1.6649 - accuracy: 0.4086
Epoch 3/50
1532/1532 [==============================] - 5s 3ms/step - loss: 1.5760 - accuracy: 0.4443
Epoch 4/50
1532/1532 [==============================] - 5s 3ms/step - loss: 1.5125 - accuracy: 0.4641
Epoch 5/50
1532/1532 [==============================] - 5s 3ms/step - loss: 1.4618 - accuracy: 0.4839
Epoch 6/50
1532/1532 [==============================] - 5s 3ms/step - loss: 1.4177 - accuracy: 0.5022
Epoch 7/50
1532/1532 [==============================] - 5s 3ms/step - loss: 1.3785 - accuracy: 0.5130
Epoch 8/50
1532/1532 [==============================] - 5s 4ms/step - loss: 1.3441 - accuracy: 0.5252
Epoch 9/50
1532/1532 [==============================] - 5s 3ms/step - loss: 1.3136 - accuracy: 0.5397
Epoch 10/50
1532/1532 [==============================] - 5s 3ms/step - loss: 1.280

In [51]:
model.evaluate(x_test_scaled, y_test_categorical)

313/313 [==============================] - 1s 2ms/step - loss: 2.1144 - accuracy: 0.4957


[2.114409923553467, 0.49570000171661377]